In [31]:
import pandas as pd
import numpy as np
import json
import os
import re
from skmultilearn.adapt import MLkNN
from sklearn.metrics import accuracy_score, recall_score, average_precision_score, f1_score
from novikova import NovikovaFeatures

nov = NovikovaFeatures()

In [133]:
robot = 'misty'

In [134]:
states_folder = f'../{robot}_filtered/states/'
files = [f for f in os.listdir(states_folder) if os.path.isfile(os.path.join(states_folder, f))]

In [135]:
def load_states_json(f, robot='cozmo'):
    states = []
    s = r''
    lines = open(f).readlines()
    
    for line in lines:
        if line == '\n':
            states.append(json.loads(s))
            s = r''
            continue
        s += line.strip()
        
    states = [json.loads(s['value']) for s in states]
    remap = []
    if robot=='cozmo':
        for s in states:
            del s['pose']
            for key in s:
        #         print(s[key])
                results = re.findall(r'\(.*?\)',s[key])
        #         for result in results:
        #         print(results)
                if len(results) > 0:
                    s[key] = float(str(results[-1][1:]).split()[0])
                elif s[key] == 'True': s[key] = True
                elif s[key] == 'False': s[key] = False
                elif 'Speed' in s[key]: s[key] = float(s[key].split()[1])  
                    
    
    elif robot=='misty':
        misty_cozmo_map = {
            'Actuator_HeadPitch' : 'head_angle',
            'Actuator_RightArm' : 'lift_height',
            'Actuator_LeftArm' : 'lift_height',
        }
#         print(states)
        remap_ = {}
        for s in states:
#             print('s', s)
            if 'Actuator_' in s['eventName']:
                if s['eventName'] in misty_cozmo_map:
                    remap_[misty_cozmo_map[s['eventName']]] = s['message']['value']
                else:
                    remap_[s['eventName']]= s['message']['value']
                remap_['are_wheels_moving'] = False
                remap_['pose_angle'] = 0
                remap_['left_wheel_speed'] = 0
                
                if len(remap_) == 7:
                    remap.append(dict(remap_))
        
        states = remap
        
    return states


# load_states_json(states_folder+files[0], robot=robot)

In [136]:
all_states = {f.replace('.json', ''):load_states_json(states_folder+f, robot=robot) for f in files}

In [137]:
np.mean([len(s) for s in all_states])

17.743119266055047

In [138]:
responses = pd.read_csv(f'../raw_data/responses_{robot}.csv')
responses['Timestamp'] = responses.Timestamp.map(lambda x: str(x))
responses = responses[responses.Timestamp.isin(all_states.keys())]
resp = {str(row[1]['Timestamp']):np.array(row[1][2:], dtype=float) for row in responses.iterrows() }

In [139]:
vectors = { ts:nov.get_vectors(all_states[ts]) for ts in all_states}

In [140]:
len(files), len(vectors), len(resp)

(545, 545, 545)

In [141]:
def softmax(x):
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x


train = list(vectors.keys())[-100:]
test = list(vectors.keys())[:-100]

X_train = np.array([vectors[v] for v in train])
X_test = np.array([vectors[v] for v in test])

y_train = np.array([resp[r]>3 for r in train], dtype=np.float32)
y_test = np.array([resp[r]>3 for r in test], dtype=np.float32)


X_train.shape, X_test.shape, y_train.shape, y_test.shape

((100, 9), (445, 9), (100, 8), (445, 8))

In [142]:
classifier_affect = MLkNN(k=5)
classifier_affect.fit(X_train, y_train)
predictions = classifier_affect.predict(X_test)
pre=np.mean([f1_score(y_test[i], predictions.toarray()[i], average='macro')  for i in range(0,len(y_test))])
acc=np.mean([accuracy_score(y_test[i], predictions.toarray()[i]) for i in range(0,len(y_test))])
print("f1 {}, acc {}".format(pre,acc))

f1 0.5351424106480287, acc 0.7485955056179775


In [144]:
train = list(vectors.keys())

if robot == 'cozmo':
    X_cozmo = np.array([vectors[v] for v in train])
    y_cozmo = np.array([resp[r]>3 for r in train], dtype=np.float32)
    
    
if robot == 'misty':
    X_misty = np.array([vectors[v] for v in train])
    y_misty = np.array([resp[r]>3 for r in train], dtype=np.float32)    
    print(X_misty.shape, y_misty.shape)

(545, 9) (545, 8)


In [151]:
X_train = X_misty
y_train = y_misty
X_test = X_cozmo
y_test = y_cozmo

classifier_affect = MLkNN(k=5)
classifier_affect.fit(X_train, y_train)
predictions = classifier_affect.predict(X_test)
pre=np.mean([f1_score(y_test[i], predictions.toarray()[i], average='macro')  for i in range(0,len(y_test))])
acc=np.mean([accuracy_score(y_test[i], predictions.toarray()[i]) for i in range(0,len(y_test))])
print("f1 {}, acc {}".format(pre,acc))

f1 0.5127091271515404, acc 0.6912705667276051


Cozmo (n_train=447, n_test=100)

f1 0.5829350366263119, acc 0.7961409395973155

Misty (n_train=445, n_test=100)

f1 0.5351424106480287, acc 0.7485955056179775

Cozmo->Misty (n_train=547, n_test=545)

f1 0.5436941040610765, acc 0.7635321100917432

Misty->Cozmo (n_train=545, n_test=547)

f1 0.5127091271515404, acc 0.6912705667276051